In [1]:
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
from transformers import ASTForAudioClassification, TrainingArguments, Trainer
import evaluate
import numpy as np


In [2]:
import pandas as pd
from datasets import Dataset, Audio, ClassLabel

# Load your CSV file into a pandas DataFrame
df = pd.read_csv('../data/processed/ast_valence.csv')

# Verify column names
print(df.head())  # Ensure columns are named correctly (e.g., 'audio' and 'label')

# Rename columns if necessary
df.rename(columns={'path_column_name': 'audio', 'class_column_name': 'label'}, inplace=True)

# Ensure the labels are integers or strings
df['label'] = df['label'].astype(str)

# Create a Dataset from the DataFrame
dataset = Dataset.from_pandas(df)

# Cast the 'audio' and 'label' columns to their respective feature types
dataset = dataset.cast_column('audio', Audio(sampling_rate=16000))
dataset = dataset.cast_column('label', ClassLabel(names=list(df['label'].unique())))

# Verify the dataset structure
print(dataset)


                                               audio  label
0  ../data/processed/1000dataset_5/specs\000RDCYi...      2
1  ../data/processed/1000dataset_5/specs\000RDCYi...      2
2  ../data/processed/1000dataset_5/specs\000RDCYi...      2
3  ../data/processed/1000dataset_5/specs\000RDCYi...      2
4  ../data/processed/1000dataset_5/specs\000RDCYi...      2


Casting the dataset:   0%|          | 0/4993 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'label'],
    num_rows: 4993
})


In [ ]:
# Load the feature extractor
feature_extractor = ASTFeatureExtractor.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')

# Define a preprocessing function
def preprocess(batch):
    # Load audio
    audio = batch['audio']
    # Extract features
    inputs = feature_extractor(audio['array'], sampling_rate=audio['sampling_rate'], return_tensors='pt')
    batch['input_values'] = inputs.input_values[0]
    return batch

# Apply the preprocessing function to the dataset
dataset = dataset.map(preprocess)


In [ ]:
# Load dataset

#
#data = "../data/processed/L1000dataset_5seg_valence.csv"

# Adjust paths to your CSV file
dataset = load_dataset("csv", data_files="../data/processed/L1000dataset_5seg_valence.csv")

# load dataset into a Dataset object
dataset = dataset["train"]

{'image_path': '../data/processed/1000dataset_5/specs\\000RDCYioLteXcutOjeweY_segment_2.png', 'class': 2}


In [ ]:
# Define features with audio and label columns
features = Features({
    "audio": Audio(),  # Define the audio feature
    "labels": class_labels  # Assign the class labels
})

dataset = Dataset.from_dict({
    "audio": dataset,
    "labels": [0, 1],  # Corresponding labels for the audio files
}, features=features)

In [10]:
from transformers import ASTFeatureExtractor

# we define which pretrained model we want to use and instantiate a feature extractor
pretrained_model = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = ASTFeatureExtractor.from_pretrained(pretrained_model)

# we save model input name and sampling rate for later use
model_input_name = feature_extractor.model_input_names[0]  # key -> 'input_values'
SAMPLING_RATE = feature_extractor.sampling_rate

In [5]:
# Preprocessing function for dataset
def preprocess_images(example):
    image = Image.open(example["image_path"]).convert("RGB")  # Open image as RGB
    example["image"] = image_transform(image)  # Apply transformations
    return example

# Apply preprocessing
dataset = dataset.map(preprocess_images)

# Rename 'class' column to 'label' (required by Trainer)
dataset = dataset.rename_column("class", "label")

# Set dataset format for PyTorch
dataset.set_format(type="torch", columns=["image", "label"])

In [7]:
# Metrics for evaluation
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    metrics = {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "f1": f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }
    return metrics

In [25]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./ast_finetuned",       # Where to save the model
    #evaluation_strategy="epoch",       # Evaluate after each epoch
    save_strategy="epoch",             # Save the model after each epoch
    learning_rate=3e-5,                # Learning rate
    per_device_train_batch_size=16,    # Train batch size
    per_device_eval_batch_size=16,     # Eval batch size
    gradient_accumulation_steps=2,     # Accumulate gradients to simulate larger batch sizes
    num_train_epochs=10,               # Number of epochs
    warmup_ratio=0.1,                  # Warmup learning rate scheduler
    logging_steps=10,                  # Log every 10 steps
    #load_best_model_at_end=True,       # Load best model at the end of training
    metric_for_best_model="f1",        # Use F1-score to evaluate best model
    save_total_limit=2,                # Limit saved models
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    #eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

# Ensure W&B is initialized with the desired project
#import wandb
#wandb.init(project="Audio_Class")

In [22]:
# check accelerate package version
import accelerate
print(accelerate.__version__)

0.28.0


In [26]:
# Train the model
trainer.train()

# create d
# Save the fine-tuned model
model.save_pretrained("./ast_finetuned")

Problem at: c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\transformers\integrations\integration_utils.py 837 setup


KeyboardInterrupt: 